# Setup

## Prepare ontology

In [1]:
%env ONTOLOGY_FILE=../ontology/mwb-ontology.owl.yaml
%env OUTPUT_DIR=../output
%env OUTPUT_VIZ=../output/Data-centric-app-demo.jpg
%env OUTPUT_OWL=../output/mwb-ontology.owl.ttl
%env OUTPUT_SHACL=../output/mwb-shapes.shacl.ttl

env: ONTOLOGY_FILE=../ontology/mwb-ontology.owl.yaml
env: OUTPUT_DIR=../output
env: OUTPUT_VIZ=../output/Data-centric-app-demo.jpg
env: OUTPUT_OWL=../output/mwb-ontology.owl.ttl
env: OUTPUT_SHACL=../output/mwb-shapes.shacl.ttl


In [2]:
!tail $ONTOLOGY_FILE

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
    tree_root: true
    attributes:
      persons:
        multivalued: true
        inlined_as_list: true
        range: Person
      companies:
        multivalued: true
        inlined_as_list: true
        range: Company


## Prepare test data

In [3]:
%env DATA_FILE=../tests/test_data/mwb-demo-data.yaml
%env OUTPUT_DATA=../output/mwb-demo-data.ttl

env: DATA_FILE=../tests/test_data/mwb-demo-data.yaml
env: OUTPUT_DATA=../output/mwb-demo-data.ttl


In [396]:
!tail $DATA_FILE

    name: Clark Kent
    age: 33
    phone: 555-555-5555
  - id: mnfy:4567
    name: Lois Lane
    age: 34
companies:
  - id: mnfy:7890
    name: ABC Inc.
    phone: 555-555-5555


## Test LinkML environment

### Validate data against ontology

In [397]:
!linkml-validate -s $ONTOLOGY_FILE $DATA_FILE

No issues found


### Export OWL ontology

In [398]:
!gen-owl $ONTOLOGY_FILE > $OUTPUT_OWL

In [399]:
!tail $OUTPUT_OWL

mnfy:name a owl:DatatypeProperty ;
    rdfs:label "name" ;
    skos:definition "name of the person" ;
    skos:inScheme <https://meaningfy.ws/data-centric-app-demo> .

mnfy:phone a owl:DatatypeProperty ;
    rdfs:label "phone" ;
    skos:inScheme <https://meaningfy.ws/data-centric-app-demo> .




### Export RDF data informed by ontology

In [400]:
!linkml-convert -s $ONTOLOGY_FILE $DATA_FILE -o $OUTPUT_DATA

In [401]:
!tail $OUTPUT_DATA


mnfy:7890 a mnfy:Company ;
    schema1:name "ABC Inc." ;
    schema1:telephone "555-555-5555" .

[] a mnfy:Container ;
    mnfy:companies mnfy:7890 ;
    mnfy:persons mnfy:1234,
        mnfy:4567 .



### Export SHACL shapes derived from ontology

In [402]:
!gen-shacl --no-metadata $ONTOLOGY_FILE > $OUTPUT_SHACL

In [403]:
!tail $OUTPUT_SHACL

        [ sh:datatype xsd:integer ;
            sh:maxCount 1 ;
            sh:maxInclusive 200 ;
            sh:minInclusive 0 ;
            sh:nodeKind sh:Literal ;
            sh:order 4 ;
            sh:path mnfy:age ] ;
    sh:targetClass schema1:Person .




### Export UML visualization of ontology

In [404]:
!echo "Exporting UML diagram.. " && gen-yuml -f jpg -d $OUTPUT_DIR $ONTOLOGY_FILE && file $OUTPUT_VIZ

Exporting UML diagram.. 
../output/Data-centric-app-demo.jpg: JPEG image data, JFIF standard 1.01, aspect ratio, density 72x72, segment length 16, baseline, precision 8, 560x215, frames 3


## Start Flur.ee database

### Get and run Flur.ee separately
You may use the Cloud instance, but keep in mind that the port we will be using is [58090](http://localhost:58090). If you experience problems signing up or logging  in, for example, seeing 404 or 403 pages, clear your cookies for the website (see [here](https://support.google.com/chrome/answer/95647?hl=en&co=GENIE.Platform%3DDesktop#zippy=%2Cdelete-cookies-from-a-site) if using Chrome).

If you prefer to "install" locally, use the Docker instance (requires you have Docker itself installed of course), which you can invoke using the [documented](https://developers.flur.ee/docs/learn/tutorial/introduction/#running-fluree) command:

```
docker run -p 58090:8090 -v `pwd`/data:/opt/fluree-server/data fluree/server:stable
```

We add the `stable` version identifier in case there are any problems with the `latest` release.

_Keep this running in a separate terminal window as it will run as a daemon/service._

In [405]:
%env FLUREE_PORT=58090
%env FLUREE_LOCAL_HTTP=http://localhost:58090
%env FLUREE_LOCAL_API=http://localhost:58090/fluree
%env FLUREE_LEDGER=mwb-fluree-poc

env: FLUREE_PORT=58090
env: FLUREE_LOCAL_HTTP=http://localhost:58090
env: FLUREE_LOCAL_API=http://localhost:58090/fluree
env: FLUREE_LEDGER=mwb-fluree-poc


### Check if database service is up and running

In [406]:
!curl -s $FLUREE_LOCAL_HTTP/index.html | grep -q swagger && echo "It's alive!"

It's alive!


### Initialize repository / ledger with constraints

With constraint on a property (`name` max 1) and initial data.

_Flur.ee does not require a UNIQUE constraint as RDF mutation on the same IRI (and therefore a resource with the same ID) is idempotent._

In [407]:
%%bash

# credit https://stackoverflow.com/questions/17029902/using-curl-post-with-variables-defined-in-bash-script-functions
create_data() {
    cat << EOF
{
  "ledger": "$FLUREE_LEDGER",
  "@context": {
    "@vocab": "https://meaningfy.ws/mapping-workbench#",
    "@base": "https://meaningfy.ws/data-centric-app-demo/",      
    "schema": "http://schema.org/",
    "sh": "http://www.w3.org/ns/shacl#"
  },
  "insert": [
      {
        "@id": "ex:UserShape",
        "@type": ["sh:NodeShape"],
        "sh:targetClass": { "@id": "schema:Person" },
        "sh:property": [
          {
            "sh:path": { "@id": "schema:name" },
            "sh:maxCount": 1
          }
        ]
      },
      {
        "@id": "parsia",
        "@type": "schema:Person",
        "schema:name": "Bijan Parsia",
        "age": 33,
        "schema:telephone": "555-555-6666"
      }
  ]
}
EOF
}

curl -s "$FLUREE_LOCAL_API/create" \
--header 'Content-Type: application/json' \
--data "$(create_data)" 
#| grep -qw mwb-test && echo "Ledger $FLUREE_LEDGER created or already exists"

{"ledger":"mwb-fluree-poc","commit":"fluree:file://mwb-fluree-poc/main/commit/70f0c02a061376a68edb248f0047357b9e458760ea7031c9709e7eb0ec086aa3.json","t":1,"tx-id":"80c46ffcb11379f15f01d2b4c4a88447da6f29e42a2019ea5933ca0b70fc4a52"}

# Test Flur.ee CRUD

## Test CREATE

Running this twice should result in an error due to constraint (SHACL) violation on cardinality of `name`.

### Create single resource

In [408]:
%%bash

create_data() {
    cat << EOF
{
  "ledger": "$FLUREE_LEDGER",
  "@context": {
     "@vocab": "https://meaningfy.ws/mapping-workbench#",
     "@base": "https://meaningfy.ws/data-centric-app-demo/",
     "schema": "http://schema.org/"
  },    
  "insert": {
    "@id": "clark",
    "@type": "schema:Person",
    "schema:name": "Clark Kent",
    "age": 33,
    "schema:telephone": "555-555-5555"
  }
}
EOF
}

curl -s "$FLUREE_LOCAL_API/transact" \
--header 'Content-Type: application/json' \
--data "$(create_data)" | python -m json.tool # credit https://stackoverflow.com/questions/352098/how-can-i-pretty-print-json-in-a-shell-script

{
    "ledger": "mwb-fluree-poc",
    "commit": "fluree:file://mwb-fluree-poc/main/commit/41acd095a6f68972cf31ff573e61e7054c6dbd7c8f3c5784285f90c158e2870b.json",
    "t": 2,
    "tx-id": "028cfcd511df4ab706585ed474d68bac8fe1dc494006ff07d6a5c531fa55f167"
}


### Create single resource with linked resources

We create another Person with linked objects at the same time, implying a composition relationship. The linked objects get random IDs and could result in duplication if this is run more than once, without any SHACL constraint on the relationship.

The linked objects could be created as siblings with their own IDs, which effectively results in an aggregation relationship.

It matters what kind of association a relationship is, as it has an effect on _cascading_ mutations (CASCADE or DETACH DELETEs in some databases). Not sure if Flur.ee has any kind of facility for such mutations, but it's good to be clear about this from the outset.

In [409]:
%%bash

create_data() {
    cat << EOF
{
  "ledger": "$FLUREE_LEDGER",
  "@context": {
     "@vocab": "https://meaningfy.ws/mapping-workbench#",
     "@base": "https://meaningfy.ws/data-centric-app-demo/",
     "schema": "http://schema.org/"
  },
  "insert": {
    "@id": "lois",
    "@type": "schema:Person",
    "schema:name": "Lois Lane",
    "age": 34,
    "history": [
        {
          "dateTime": "2022-12-20",
          "location": "ski basin"
        },
        {
          "dateTime": "2022-12-21",
          "location": "Target"
        }
    ]
  }
}
EOF
}

curl -s "$FLUREE_LOCAL_API/transact" \
--header 'Content-Type: application/json' \
--data "$(create_data)" | python -m json.tool

{
    "ledger": "mwb-fluree-poc",
    "commit": "fluree:file://mwb-fluree-poc/main/commit/49c96c00d98354feea959ccdd197c2e16a4776cb140f9e3d314ffb02a7cba54b.json",
    "t": 3,
    "tx-id": "ed24dbcdf6a919e9dc7e94581a28d300e4fc58511e275d4838b53166b6534170"
}


## Test UPDATE

### Update single resource with new linked resources
Join a list of `history` items with `ex:clark` ("Clark Kent") in an aggregation fashion. The linked objects, however, have no IDs of their own, and there was no constraint set on `history`, so will result in duplication if this is run more than once.

In [410]:
%%bash

create_data() {
    cat << EOF
{
  "ledger": "$FLUREE_LEDGER",
  "@context": {
     "@vocab": "https://meaningfy.ws/mapping-workbench#",
     "@base": "https://meaningfy.ws/data-centric-app-demo/",
     "schema": "http://schema.org/"
  },
  "insert": {
    "@id": "clark",
    "history": [
        {
          "day": "2022-12-20",
          "location": "ski basin"
        },
        {
          "day": "2022-12-21",
          "location": "Target"
        }
    ]
  }
}
EOF
}

curl -s "$FLUREE_LOCAL_API/transact" \
--header 'Content-Type: application/json' \
--data "$(create_data)" | python -m json.tool

{
    "ledger": "mwb-fluree-poc",
    "commit": "fluree:file://mwb-fluree-poc/main/commit/9df524bbd84b065ef5c28a6f5c1fbd13b0e50a8267d1653ecb8d186e31c8708e.json",
    "t": 4,
    "tx-id": "5f5e1fdcac5e962d2479612764be91fdd7d12dcefb64214e9298b29dcc9038e9"
}


## Test DELETE

**THIS IS NOT WORKING AS EXPECTED - RESOURCE IS STILL THERE (SEE READ RESULTS).**

In [411]:
%%bash

create_data() {
    cat << EOF
{
  "ledger": "$FLUREE_LEDGER",
  "@context": {
     "@vocab": "https://meaningfy.ws/mapping-workbench#",
     "@base": "https://meaningfy.ws/data-centric-app-demo/",
     "schema": "http://schema.org/"
  },
  "where": {
    "@id": "parsia",
    "?p": "?o"
  },
  "delete": {
    "@id": "parsia",
    "?p": "?o"
  }
}
EOF
}

curl -s "$FLUREE_LOCAL_API/transact" \
--header 'Content-Type: application/json' \
--data "$(create_data)" | python -m json.tool

{
    "ledger": "mwb-fluree-poc",
    "commit": "fluree:file://mwb-fluree-poc/main/commit/5e22a4516b53b52c6ee20de5d699bfc42794dc49915ba184f05a1b34c792be52.json",
    "t": 5,
    "tx-id": "b7e7655a6b0cfbeca110f7b2fa01a1f34ff11ee7d546ff253b72b5e27cf5e81b"
}


## Test READ

In [412]:
%%bash

create_data() {
    cat << EOF
{
  "from": "$FLUREE_LEDGER",
  "@context": {
     "@vocab": "https://meaningfy.ws/mapping-workbench#",
     "@base": "https://meaningfy.ws/data-centric-app-demo/",
     "schema": "http://schema.org/"
  },
  "select": {
    "?s": ["*", {"history": ["*"]}]
  },
  "where": {
    "@id": "?s",
    "@type": "schema:Person"
  }
}
EOF
}

curl -s "$FLUREE_LOCAL_API/query" \
--header 'Content-Type: application/json' \
--data "$(create_data)" | python -m json.tool

[
    {
        "@id": "lois",
        "@type": "schema:Person",
        "age": 34,
        "history": [
            {
                "@id": "_:f211106232532998",
                "dateTime": "2022-12-20",
                "location": "ski basin"
            },
            {
                "@id": "_:f211106232532999",
                "dateTime": "2022-12-21",
                "location": "Target"
            }
        ],
        "schema:name": "Lois Lane"
    },
    {
        "@id": "clark",
        "@type": "schema:Person",
        "schema:telephone": "555-555-5555",
        "age": 33,
        "history": [
            {
                "@id": "_:f211106232533000",
                "location": "Target",
                "day": "2022-12-21"
            },
            {
                "@id": "_:f211106232533001",
                "location": "ski basin",
                "day": "2022-12-20"
            }
        ],
        "schema:name": "Clark Kent"
    },
    {
        "@id": "parsia",
   